In [1]:
from f_data_prep import *
from f_drosophila_infer import *
from f_train import *
import multiprocessing
import os

In [2]:
# ratio = np.array([int(5470*i) for i in [0.2,0.22,0.24,0.26,0.28,0.3,0.33,0.36,0.4,0.45,0.5,0.55,0.6]])

In [3]:
# n = np.round(ratio/0.9)
# n

array([1216., 1337., 1458., 1580., 1701., 1823., 2006., 2188., 2431.,
       2734., 3039., 3342., 3647.])

In [2]:
data_all = np.loadtxt('../data_complete.txt')
all_bin = np.vsplit(data_all, 6)
all_init = np.vstack([all_bin[i] for i in range(5)])
all_diff = np.vstack([all_bin[i+1]-all_bin[i] for i in range(5)])

In [3]:
def make_quad(X):
    quad = np.zeros((int(X.shape[0]), int(X.shape[1] + (X.shape[1]*(X.shape[1]-1))/2)))
    quad[:, :X.shape[1]] = np.copy(X)
    col = 99
    for i in range(X.shape[1]-1):
        for j in range(i+1, X.shape[1]):
            quad[:,col] = (X[:,i]*X[:,j])
            col += 1
    return quad

quad = make_quad(all_init)

with open('./pickles/validation_cells.pkl', 'rb') as f:
    cells_v = pickle.load(f)
    
val = np.hstack([cells_v+(6078*i) for i in range(5)])
cells_tt = np.delete(range(6078), cells_v)
tr_te = np.hstack([cells_tt+(6078*i) for i in range(5)])

X_v = all_init[val]
quad_v = quad[val]
y_v = all_diff[val]
quad_tt = quad[tr_te]
y_tt = all_diff[tr_te]

In [4]:
# quad_tr = []
# y_tr = []
# quad_te = []
# y_te = []

# kfold = KFold(n_splits=10, shuffle=False)
# for (i, (cell_tr, cell_te)) in enumerate((kfold.split(range(5470)))):
# #     print(i, cell_tr, cell_te)
#     te = np.hstack([cell_te+(5470*i) for i in range(5)])
#     tr = np.hstack([cell_tr+(5470*i) for i in range(5)])
#     np.save('./numpy_files/quad_tr_%s' % i, quad_tt[tr])
#     np.save('./numpy_files/y_tr_%s' % i, y_tt[tr])
#     np.save('./numpy_files/quad_te_%s' % i, quad_tt[te])
#     np.save('./numpy_files/y_te_%s' % i, y_tt[te])

In [7]:
def make_quad(X):
    quad = np.zeros((int(X.shape[0]), int(X.shape[1] + (X.shape[1]*(X.shape[1]-1))/2)))
    quad[:, :X.shape[1]] = np.copy(X)
    col = 99
    for i in range(X.shape[1]-1):
        for j in range(i+1, X.shape[1]):
            quad[:,col] = (X[:,i]*X[:,j])
            col += 1
    return quad

quad = make_quad(all_init)

with open('./pickles/validation_cells.pkl', 'rb') as f:
    cells_v = pickle.load(f)
    
val = np.hstack([cells_v+(6078*i) for i in range(5)])

titrate = [20,22,24,26,28,30,33,36,40,45,50,55,60]
ind = 0
for cell_n in n:
    per = int(titrate[ind])
    cells_tt = np.random.choice(np.delete(range(6078), cells_v), size=int(cell_n), replace=False)
    tr_te = np.hstack([cells_tt+(6078*i) for i in range(5)])

    quad_tt = quad[tr_te]
    y_tt = all_diff[tr_te]

    kfold = KFold(n_splits=10, shuffle=False)
    for (j, (cell_tr, cell_te)) in enumerate((kfold.split(range(int(cell_n))))):
    #     print(i, cell_tr, cell_te)
        te = np.hstack([cell_te+(int(cell_n)*i) for i in range(5)])
        tr = np.hstack([cell_tr+(int(cell_n)*i) for i in range(5)])
        np.save('./numpy_files/titrate_%s/quad_tr_%s' % (per,j), quad_tt[tr])
        np.save('./numpy_files/titrate_%s/y_tr_%s' % (per,j), y_tt[tr])
        np.save('./numpy_files/titrate_%s/quad_te_%s' % (per,j), quad_tt[te])
        np.save('./numpy_files/titrate_%s/y_te_%s' % (per,j), y_tt[te])
    ind += 1